In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from  torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.optim.lr_scheduler import StepLR
import torch.backends.cudnn as cudnn
import torch.optim as optim 
from torch.optim import lr_scheduler 
from tempfile import TemporaryDirectory
import os 
import time

from torch.utils.data import random_split
%matplotlib inline

In [2]:
batch_size=128
val_size = 10000

torch.manual_seed(1)

dataset = MNIST(root='data/', download=True, transform=ToTensor())

train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)

In [3]:
train_mean = 0.
train_std = 0.
for images, _ in train_loader:
    batch_samples = images.size(0) 
    images = images.view(batch_samples, images.size(1), -1)

    train_mean += images.mean(2).sum(0)
    train_std += images.std(2).sum(0)

train_mean /= len(train_loader.dataset)
train_std /= len(train_loader.dataset)

print('Mean: ', train_mean)
print('Std: ', train_std)

Mean:  tensor([0.1307])
Std:  tensor([0.3016])


In [46]:
dataset = MNIST(root='data/', download=True, transform=transforms.Compose([
                        #transforms.Grayscale(3),
                        transforms.Resize(256), 
                        transforms.CenterCrop(224), 
                        transforms.RandomRotation((-7.0,7.0),fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1308,), (0.3016,))
                    ]))
test_dataset = MNIST(root='data/', train=False,transform=transforms.Compose([
                        #transforms.Grayscale(3),
                        transforms.Resize(256), transforms.CenterCrop(224),  
                        transforms.ToTensor(), 
                        transforms.Normalize((0.1308,), (0.3016,))
                    ]))

In [15]:
torch.manual_seed(1)
# split to training and validation 
val_size = 10000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

(50000, 10000)

In [47]:
batch_size=128

train_loader = DataLoader(dataset, batch_size, shuffle=True, num_workers=4, pin_memory=True)
#val_loader = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size*2, num_workers=4, pin_memory=True)

RuntimeError: DataLoader worker (pid(s) 8576, 1080, 12756, 1656) exited unexpectedly

In [48]:
import matplotlib.pyplot as plt 
import numpy as np 

def imshow(img):
    img = img / 2 + 0.5 
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(train_loader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))


RuntimeError: DataLoader worker (pid(s) 17752, 19048, 15668) exited unexpectedly

In [44]:
print(images.shape)
images.unsqueeze(-1)
images = images.expand(128, 3, 224, 224)
print(images.shape)

torch.Size([128, 3, 224, 224])
torch.Size([128, 3, 224, 224])


In [7]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [10]:
alex_net = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)
alex_net

Using cache found in C:\Users\Владелец/.cache\torch\hub\pytorch_vision_v0.6.0
C:\Users\Владелец\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Владелец\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [11]:
alex_net.classifier[4] = nn.Linear(4096, 1024)
alex_net.classifier[6] = nn.Linear(1024, 10)
alex_net

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [45]:
alex_net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alex_net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):
    running_loss = 0. 
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        print(inputs.shape)
        optimizer.zero_grad()  
        inputs.unsqueeze(-1)
        inputs = images.expand(128, 3, 224, 224)
        output = alex_net(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2000 == 1999: # print every 2000 minibatches 
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0. 
print("done")

RuntimeError: DataLoader worker (pid(s) 460) exited unexpectedly

Epoch 0/24
----------


NameError: name 'dataloaders' is not defined